### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from tabulate import tabulate
import targets

### Options

In [ ]:
experiment = 'RainingClumps'
catalog_name = 'UVISTA-PLUS'
field = 'COSMOS'
target_id = 145441 # COSMOS2015 489594

options = {
    'instrument'        : 'GNIRS-LR_IFU-Altair-LGSAO',
    'resolving_power'   : 7200,
    'obs_epoch'         : 2025.1068, # 2025-02-08
    'lines'             : ['NIIa', 'Ha', 'NIIb'],
    'default_dz'        : 3e-4, # deprectate
    'default_dispersion': 200.0, # deprectate
}

reload = False

### Load Target

In [ ]:
if reload or not targets.exists_details(experiment):
    galaxies, lines, options = targets.append_details(options, [catalog_name, [target_id]])
    targets.save_details(experiment, galaxies, lines, options)
    del galaxies, lines, options

In [ ]:
galaxy, lines, options = targets.load_details(experiment, id=target_id)

### Details on Selected Target

In [ ]:
targets.print_summary(galaxy, lines, options, line_name='Ha')

### Sky Transmission

In [ ]:
targets.plot_sky_transmission(galaxy, lines, options)

### Sky Background

In [ ]:
targets.plot_sky_background(galaxy, lines, options)

### Nearby Stars

In [ ]:
stars = targets.find_nearby_stars(galaxy['ra'], galaxy['dec'], options)
if len(stars) == 0:
    print("No stars found")
else:
    display(tabulate(targets.fix_gaia_ids_for_display(stars), headers=stars.colnames, tablefmt='html'))

    ngs_id = 3836241737698207488

    stars_idx = np.where(stars['gaia_id'] == ngs_id)[0][0]
    print(f"Obs Epoch ({options['obs_epoch']}): ra={stars['obs_ra'][stars_idx]:.10f} dec={stars['obs_dec'][stars_idx]:.10f}")
    del stars_idx

### Available Images

In [ ]:
catalog_filters = targets.get_catalog_filters(field, galaxy)
if catalog_filters is None or len(catalog_filters) == 0:
    print("No catalog images found")
else:
    display(tabulate(catalog_filters, headers=catalog_filters.colnames, tablefmt='html'))

catalog_filter_index = 1

In [ ]:
space_filters = targets.get_space_filters(field, galaxy)
if space_filters is None or len(space_filters) == 0:
    print('No space imaging found')
else:
    display(tabulate(space_filters, headers=space_filters.colnames, tablefmt='html'))

space_filter_index = 1

### Cut-Outs

In [ ]:
catalog_cutout = targets.get_catalog_cutout(field, galaxy, options, catalog_filters['filter'][catalog_filter_index])
decon_cutout = targets.get_decon_cutout(field, galaxy, return_none_if_missing=True)
space_cutout = targets.get_space_cutout(field, galaxy, space_filters['filter'][space_filter_index])

In [ ]:
if decon_cutout is None:
    targets.plot_galaxy_cutouts(galaxy, options, [catalog_cutout, space_cutout])
else:
    targets.plot_galaxy_cutouts(galaxy, options, [catalog_cutout, decon_cutout, space_cutout])

In [ ]:
targets.plot_galaxy_cutouts(galaxy, options, catalog_cutout, cbar=True, skip_norm=True)

### Simulate IFU

In [ ]:
ifu = targets.get_ifu_position(galaxy, options,
    pa=90, # degrees
    sp=1,  # spaxels
    sq=2,  # spaxels
)

targets.plot_galaxy_cutouts(galaxy, options, space_cutout, ifu=ifu,
    resolution=0.15, # arcsec
    segmap=True,
    ifu_grid=True
)

In [ ]:
targets.plot_dithers(galaxy, options, space_cutout, ifu,
    sp=[-2.5,  2.0, -2.5, 2.0],
    sq=[ 5.5, -5.0, -5.0, 5.5],
    resolution=0.15, # arcsec
    segmap=True,
    ifu_grid=True
)

### Finder Chart

In [ ]:
targets.plot_finder_chart(catalog_cutout, options, ifu=ifu, stars=stars, ngs_id=ngs_id,
    vmin=0,
    vmax=50
)

### Check Blind Offset

In [ ]:
if len(stars) > 0:
    targets.compare_star_centroid(catalog_cutout, stars, ngs_id, options, ifu=ifu,
        radius=2.5 # arcsec
    )